# Step 0: Segment original FOS dataset to the 10s video clips

I have modeified "seg_to_10s_av_clips.py", just run the code "seg_to_10s_av_clips.py" to segment the original FOS dataset to the 10s video clips. Keep the corresponding labels in the CSV file.

# Step 1: Extract audio track and image frames from the video

## Step 1.1: build the 10s video clips list in CSV format

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
df_original_dataset = pd. read_csv(r'10s_av_clips_with_labels.csv')
df_paths_clip = pd.DataFrame([os.path.normpath(path) for path in df_original_dataset['path'].values], columns=['path'])
df_paths_clip.to_csv('paths_clip.csv', header=False, index=False)
df_paths_clip = pd.read_csv('paths_clip.csv')
df_paths_clip

,C:\Users\36394\Study\GWU\PHD in Biomedical Engineer\Research\FOS\Autism_dataset\Data_10s_clips\20th_BAM_Hospital_Playtime_New Toys_0.mp4
0,C:\Users\36394\Study\GWU\PHD in Biomedical Eng...
1,C:\Users\36394\Study\GWU\PHD in Biomedical Eng...
2,C:\Users\36394\Study\GWU\PHD in Biomedical Eng...
3,C:\Users\36394\Study\GWU\PHD in Biomedical Eng...
4,C:\Users\36394\Study\GWU\PHD in Biomedical Eng...
...,...
8102,C:\Users\36394\Study\GWU\PHD in Biomedical Eng...
8103,C:\Users\36394\Study\GWU\PHD in Biomedical Eng...
8104,C:\Users\36394\Study\GWU\PHD in Biomedical Eng...
8105,C:\Users\36394\Study\GWU\PHD in Biomedical Eng...


## Step 1.2: extract audio track and image frames from the video
I have modified the
"\src\preprocess\extract_audio.py" and "\src\preprocess\extract_video_frame.py"
to fit our FOS dataset.
In the terminal, run the following command to extract audio track and image frames from the video:
```bash
python src/preprocess/extract_audio.py
  -input_file_list "C:\Users\36394\PycharmProjects\cav-mae\zzh_code\paths_clip.csv"
  -target_fold "C:\Users\36394\Study\GWU\PHD in Biomedical Engineer\Research\FOS\Autism_dataset\cav_mae_processed_data\audio_10s"

python src/preprocess/extract_video_frame.py
  -input_file_list "C:\Users\36394\PycharmProjects\cav-mae\zzh_code\paths_clip.csv"
  -target_fold "C:\Users\36394\Study\GWU\PHD in Biomedical Engineer\Research\FOS\Autism_dataset\cav_mae_processed_data\RGB_10_frames"
```

# Step 2: Build a FOS label csv and json files for the FOS dataset.

## Step 2.1: Build a FOS label csv.

In [21]:
# Find all the unique labels (ISs) in the FOS dataset
import pandas as pd
import numpy as np
from ast import literal_eval
df_FOS_10s_with_label = pd.read_csv(r'10s_av_clips_with_labels.csv')
list_labels = []
for i in df_FOS_10s_with_label.labels:
    list_labels.extend(literal_eval(i))
list_labels = np.unique(list_labels)
list_labels

array(['AD', 'AV', 'Aff_child', 'Aff_parent', 'C+', 'C-', 'CP', 'EA',
       'Int_child', 'Int_parent', 'MI', 'NC', 'O', 'P', 'PN', 'Q+', 'Q-',
       'S+', 'S-', 'SI+', 'SI-', 'VI+', 'VI-'], dtype='<U10')

In [25]:
dict_FOS_labels = {
    'AD': 'Adersive demand',
    'AV': 'Appropriate verbal interactions',
    'Aff_child': 'Children affection',
    'Aff_parent': 'Parent affection',
    'C+': 'Positive contact',
    'C-': 'Negative contact',
    'CP': 'Complaint',
    'EA': 'Engaged activity of play',
    'Int_child': 'Children interrupt',
    'Int_parent': 'Parent interrupt',
    'MI': 'Mild instruction',
    'NC': 'Non-compliance',
    'O': 'Oppositional',
    'P': 'Praise',
    'PN': 'Physical negative',
    'Q+': 'Positive question',
    'Q-': 'Negative question',
    'S+': 'Positive social attention',
    'S-': 'Negative social attention',
    'SI+': 'Positive specific instruction',
    'SI-': 'Negative specific instruction',
    'VI+': 'Positive vague instruction',
    'VI-': 'Negative vague instruction'
}
dict_FOS_labels

{'AD': 'Adersive demand',
 'AV': 'Appropriate verbal interactions',
 'Aff_child': 'Children affection',
 'Aff_parent': 'Parent affection',
 'C+': 'Positive contact',
 'C-': 'Negative contact',
 'CP': 'Complaint',
 'EA': 'Engaged activity of play',
 'Int_child': 'Children interrupt',
 'Int_parent': 'Parent interrupt',
 'MI': 'Mild instruction',
 'NC': 'Non-compliance',
 'O': 'Oppositional',
 'P': 'Praise',
 'PN': 'Physical negative',
 'Q+': 'Positive question',
 'Q-': 'Negative question',
 'S+': 'Positive social attention',
 'S-': 'Negative social attention',
 'SI+': 'Positive specific instruction',
 'SI-': 'Negative specific instruction',
 'VI+': 'Positive vague instruction',
 'VI-': 'Negative vague instruction'}

In [31]:
# Create label dataframe
df_FOS_label = pd.DataFrame({
    'index': list(range(0, len(dict_FOS_labels))),  # 0-based index
    'mid': list(dict_FOS_labels.keys()),  # The IS code in the FOS dataset
    'display_name': list(dict_FOS_labels.values())  # The IS in the FOS dataset
})
# Store and display the label dataframe
df_FOS_label.to_csv('FOS_label.csv', index=False)
df_FOS_label

,index,mid,display_name
0,0,AD,Adersive demand
1,1,AV,Appropriate verbal interactions
2,2,Aff_child,Children affection
3,3,Aff_parent,Parent affection
4,4,C+,Positive contact
5,5,C-,Negative contact
6,6,CP,Complaint
7,7,EA,Engaged activity of play
8,8,Int_child,Children interrupt
9,9,Int_parent,Parent interrupt


## Step 2.2: Build a FOS label json.

In [37]:
import json, os

# Change the data dictionary if we used server to run the code
dir_10_frames = r"C:\Users\36394\Study\GWU\PHD in Biomedical Engineer\Research\FOS\Autism_dataset\cav_mae_processed_data\RGB_10_frames"
dir_audio = r'C:\Users\36394\Study\GWU\PHD in Biomedical Engineer\Research\FOS\Autism_dataset\cav_mae_processed_data\audio_10s'

# Create the empty json dataset
json_FOS_dataset = []

# Convert the 10s video clips with labels to the json format
df_FOS_10s_with_label = pd.read_csv(r'10s_av_clips_with_labels.csv')
for index, row in df_FOS_10s_with_label.iterrows():
    # Get the video id, labels and the path of the audio track
    video_id = os.path.basename(row['path'][:-4])
    labels = literal_eval(row['labels'])
    labels = ', '.join(labels)
    path_audio = os.path.join(dir_audio, video_id + '.wav')

    # Create the new entry for the json file
    new_entry = {
        'video_id': video_id,
        'wav': path_audio,
        'video_path': dir_10_frames,
        'labels': labels
    }

    # Append the new entry to the json dataset
    json_FOS_dataset.append(new_entry)
json_FOS_dataset = {'data': json_FOS_dataset}

# save the json FOS dataset to a json file
file_path = 'FOS_dataset.json'  # 指定文件保存路径
with open(file_path, 'w') as file:
    json.dump(json_FOS_dataset, file, indent=4)

